# Pandas и большие файлы

In [ ]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [ ]:
df = pd.read_csv('sales_db.csv')
df

In [ ]:
df.groupby('user_id').max()[['cost']]

In [ ]:
previous_user = None
user_data = []

with open('sales_db.csv') as f:
    f.readline()
    # previous_user = f.readline().strip(',')[0]
    for i, line in enumerate(f):
        user_id, category, cost = line.strip().split(',')
        cost = int(cost)
        print(user_id, category, cost)
        
        if previous_user:
            if previous_user == user_id:
                pass
            else:
#                 print(user_data)
#                 print(previous_user, max([x[2] for x in user_data]))
                print(previous_user, sorted(user_data, key=lambda x: -x[2])[0][1])
                print('\n')
                
                user_data = []
                previous_user = user_id
        else:
            previous_user = user_id
        
        user_data.append([user_id, category, cost])
            
print(user_data)
print(previous_user, max([x[2] for x in user_data]))
print(previous_user, sorted(user_data, key=lambda x: -x[2])[0][1])

# Объединение датафреймов

### Данные со слайдов

In [ ]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

In [ ]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

In [ ]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

In [ ]:
visits.groupby('user_id').count().reset_index()

In [ ]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

In [ ]:
purchases_pivot.reset_index()

In [ ]:
visits_grouped.join(purchases_pivot)

### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [ ]:
visits_grouped.join(purchases_pivot, how='left')

In [ ]:
visits_grouped.join(purchases_pivot, how='left', on=['user_id'])

In [ ]:
visits_grouped.join(purchases_pivot, how='left')

In [ ]:
visits_grouped.merge(purchases_pivot, how='left', left_on=['user_id', 'source'], 
                     right_on=['user_name', 'category'])

### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [ ]:
visits_grouped.join(purchases_pivot, how='right')

### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [ ]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [ ]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

In [ ]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

In [ ]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

In [ ]:
(clients
 .merge(earnings, how='left', on='id')  # объединение с доходами
 .merge(spending, how='left', on='id')
)

In [ ]:
joined = clients.merge(earnings, how='left', on='id')
result = joined.merge(spending, how='left', on='id')

result[result.debit < result.credit]

In [ ]:
result = clients.merge(earnings, how='left') 
result = result.merge(spending, how='left') 

result[result.credit > result.debit]

### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [ ]:
visits_grouped.join(purchases_pivot, how='inner')

### Outer join
Оставляет все строчки

In [ ]:
visits_grouped.join(purchases_pivot, how='outer')

# Конкатенация таблиц

In [ ]:
'123' + '456'

In [ ]:
[1, 2, 3] + [4, 5, 6]

In [ ]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [ ]:
a

In [ ]:
b

In [ ]:
pd.concat([a, b])

In [ ]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

In [ ]:
import os

In [ ]:
df = pd.DataFrame()

for file in os.listdir('folder_with_data'):
    # df_temp = pd.read_excel(file)
    df_temp = pd.read_csv(file)
    df.concat([df, df_temp])

### Дубликаты при объединении таблиц

In [ ]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

In [ ]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

In [ ]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

In [ ]:
movies.drop_duplicates(subset = 'movieId', keep = 'last', inplace = True)
movies.head()

In [ ]:
ratings.merge(movies, how = 'left', on = 'movieId')

In [ ]:
ratings.merge(movies, how = 'right', on = 'movieId')

### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [ ]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

In [ ]:
visits.groupby('date').sum().reset_index().merge(orders, on='date', how='left')

In [ ]:
temp = (visits
 .pivot_table(index='date', columns='source', values='visits', aggfunc='sum')
 .reset_index()
 .fillna(0)
 .merge(orders, on='date', how='left')
)
temp

In [ ]:
temp['visits'] = temp['organic'] + temp['paid']
temp

In [ ]:
temp = visits.merge(orders, on='date', how='left')
temp

In [ ]:
temp['']

In [ ]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

### Оптимизация хранения данных

In [ ]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [ ]:
ratings.info()

In [ ]:
movies.info()

In [ ]:
Nan

In [ ]:
joined.head()

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
len(ratings) == len(joined)

In [ ]:
joined.to_csv('joined_ratings.csv', index=False)

In [ ]:
joined.head()

In [ ]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [ ]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

### Какой жанр имеет самые высокие рейтинги?

In [ ]:
import numpy as np

In [ ]:
genres = ['Drama', 'Action', 'Thriller']

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

In [ ]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

### Считаем рейтинг жанров

In [ ]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [ ]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [ ]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

In [ ]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [ ]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

# ========== Домашняя работа ==========

In [1]:
import pandas as pd
import datetime as dt

### Домашнее задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
"""Этот код два дня писал... надеюсь, что правильно..."""

ratings = pd.read_csv('ml-latest-small/ratings.csv')

"""Создаем новый DataFrame на основе фильтра"""
filtered = ratings.groupby(['userId'])['userId'].count() > 100
filteredDT = pd.DataFrame(filtered.values, filtered.index).reset_index()
filteredDT.columns = ['userId','filtered_100']

"""Считаем разницу между максимальной датой и минимальной в рамках одного пользователя"""
time_delta = ratings.groupby('userId').timestamp.max() - ratings.groupby('userId').timestamp.min()

"""Создаем новый DataFrame с данными по разницей между датами"""
time_deltaDT = pd.DataFrame(time_delta.values, time_delta.index).reset_index()
time_deltaDT.columns = ['userId', 'delta_time']

"""Соединяем исходный DataFrame и с двумя новыми DataFrame"""
ratings = ratings.merge(filteredDT, on='userId', how='left').merge(time_deltaDT, on='userId', how='left')

"""Считаем среднее время жизни пользователей (по всем пользователям, выставившим более 100 оценок)"""
delta_time_mean = dt.datetime.fromtimestamp(ratings[ratings['filtered_100'] == True].delta_time.mean()) # datetime.datetime(1972, 11, 25, 22, 41, 37, 971225)
delta_time_mean = delta_time_mean - dt.datetime(1970, 1, 1, 0, 0, 0) #Вычитаем 1970 годОВ из среднего значения
print(f'Среднее время жизни пользователей, выставивших более 100 оценок: {delta_time_mean.days} дн. (или примерно {round(delta_time_mean.days/365, 2)} года).')

Среднее время жизни пользователей, выставивших более 100 оценок: 1059 дн. (или примерно 2.9 года).


### Домашнее задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [3]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [4]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [5]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [6]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [7]:
table_2 = (client_base # Содержит все ID, поэтому присоединяем к ней
           .merge(rzd, on='client_id', how='left')
           .merge(auto, on='client_id', how='left')
           .merge(air, on='client_id', how='left')
          ) #Создали таблицу со всеми данными

table_1 = table_2[['client_id', 'rzd_revenue', 'auto_revenue', 'air_revenue']]

In [8]:
table_1.fillna(0) # Решил, что лучше заполнить NaN нулями, так как это финансовая таблица

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [9]:
table_2.fillna(0)  # Решил, что лучше заполнить NaN нулями, так как это финансовая таблица

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


### Домашнее задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

#### Решение

Даны следующие дынные по каждому пользователю:

– индификатор пользователя {'user_id': ['ID000001']}

– широта местоположения пользователя {'latitude': [55.7522200]}

– долгота местоположения пользователья {'longitude': [37.6155600]}

– время определения местоположения польвателя {'unixtime': [1594126693]}

In [10]:
data = pd.DataFrame(
    {'user_id': ['ID000001'], 
     'latitude': [55.7522200], 
     'longitude': [37.6155600], 
     'unixtime': [1594126693]}
)
data

,user_id,latitude,longitude,unixtime
0,ID000001,55.75222,37.61556,1594126693


В таблицу визитов и покупок новые данные добавляем посредством привязки через "user_id" (предполагаем, что в общей таблице он такой же). Объединять таблицы будем так, чтобы в основной таблице данные не пропали (выбираем соответсвующий тип объядинения, скорей всего left (слево основная таблица)), потому что нам не особо нужны данные, которые не будут в итоге привязаны к покупкам.

Если "визит" – это дата, то дополнительная привязка – по дате визита и дате определения координат пользователя.

Существенная разница координат по идеи не должна как-то сказываться, так как по координатам нет привязки; но если в одну дату координаты пользователя существенно менялись (допустим, более, чем на 10 км или в каких-то других измерениях), то можно подумать над исключением таких данных (или их пометки в отдельном столбце, что они могут быть не достаточно надежными) (не исключено, что пользователь заходил через VPN);
если координаты существенно менялись в разные даты, то возможно пользователь просто перемещался в эти дни и это нормально (или по-прежнему заходил через VPN каждый раз).

Из координат можно получить: страну, регион, населенный пункт – с высокой степенью точности; дом – предположительно. Наверняка есть сервисы, которые отдают все эти данные по координатам (наверное, это: yandex, google, openstreetmap и т.д).